In [20]:

# Добавление каталога в sys.path
import sys
sys.path.append('/home/ivant/DS_bootcamp/Phase3_lec_hw/Week1/Day1/HW/fastapi-streamlit/api')

import PIL
from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel
from api.utils.model_func import class_id_to_label, load_model, transform_image
import cv2
import numpy as np
import base64
from io import BytesIO
from fastapi.responses import JSONResponse
from PIL import Image
import io
from pathlib import Path
from ultralytics import YOLO

In [2]:
model = None 
app = FastAPI()

In [3]:
# Create class of answer: only class name 
class ImageClass(BaseModel):
    prediction: str

class TextClass(BaseModel):
    text: str

In [17]:
# Load model at startup
@app.on_event("startup")
def startup_event():
    global model
    model = load_model()

/tmp/ipykernel_326709/3980485384.py:2: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [5]:
# Функция для рисования bounding boxes
def draw_boxes(image, detections):
    if 'boxes' in detections:
        for box in detections['boxes']:
            # Распаковываем значения из кортежа
            x1, y1, x2, y2, conf, cls = box
            # Преобразуем координаты в целые числа, если это еще не сделано
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            # Рисуем прямоугольник на изображении
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # Опционально: добавление текста к bounding box
            cv2.putText(image, f'Conf: {conf:.2f}, Class: {cls}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
    return image

In [6]:
@app.post('/detect')
async def detect_objects(file: UploadFile = File(...)):
    # Чтение изображения
    image_bytes = await file.read()
    image = np.frombuffer(image_bytes, dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    # Вызов вашей модели для получения детекций
    # Здесь предполагается, что detections - это список словарей с координатами bounding boxes
    detections = model.predict(image)

    # Рисование bounding boxes на изображении
    # test_detections = {'boxes': [(100, 100, 200, 200, 0.9, 1)]}
    image_with_boxes = draw_boxes(image, detections)

    # Конвертация обратно в формат, пригодный для отправки через HTTP
    _, buffer = cv2.imencode('.jpg', image_with_boxes)
    img_str = base64.b64encode(buffer).decode('utf-8')

    return JSONResponse(content={"image": img_str})

In [7]:
test_img = '/home/ivant/Downloads/Test_images/1. Ships/1f8b6650c_jpg.rf.2f6c610e2ad91d0bc71134043c532e0e.jpg'

In [9]:
test_image_path = Path('/home/ivant/Downloads/Test_images/1. Ships/1f8b6650c_jpg.rf.2f6c610e2ad91d0bc71134043c532e0e.jpg')
assert test_image_path.exists(), "Test image does not exist."

In [23]:
# Путь к файлу конфигурации и весам модели
weights_path = 'api/utils/best_yolo_weights.pt'

In [27]:
def load_model():
    '''
    Returns YOLO8 model with trained weights
    '''

    '''
    Возвращает модель YOLO с загруженными весами.
    '''

    # Создание экземпляра модели без загрузки весов
    model = YOLO(weights_path)

    # Загрузка весов модели
    # model.load(weights_path)

    return model

In [28]:
model = load_model()

In [29]:
# Загрузка и предсказание
image = cv2.imread(str(test_image_path))
detections = model.predict(image)


0: 640x640 2 ships, 16.6ms
Speed: 1.9ms preprocess, 16.6ms inference, 419.2ms postprocess per image at shape (1, 3, 640, 640)


In [36]:
detections

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'ship'}
 obb: None
 orig_img: array([[[59, 51, 21],
         [76, 68, 38],
         [90, 82, 52],
         ...,
         [66, 58, 28],
         [80, 72, 42],
         [90, 82, 52]],
 
        [[63, 55, 25],
         [76, 68, 38],
         [88, 80, 50],
         ...,
         [67, 59, 29],
         [77, 69, 39],
         [83, 75, 45]],
 
        [[66, 58, 28],
         [75, 67, 37],
         [83, 75, 45],
         ...,
         [70, 62, 32],
         [74, 66, 36],
         [74, 66, 36]],
 
        ...,
 
        [[81, 73, 44],
         [77, 69, 40],
         [73, 65, 36],
         ...,
         [67, 55, 31],
         [72, 60, 36],
         [74, 62, 38]],
 
        [[80, 72, 43],
         [77, 69, 40],
         [76, 68, 39],
         ...,
         [67, 55, 31],
         [68, 56, 32],
         [67, 55, 31]],
 
        [[79, 71, 42],
      

In [31]:
# Функция для рисования bounding boxes
def draw_boxes(image, detections):
    if 'boxes' in detections:
        for box in detections['boxes']:
            # Распаковываем значения из кортежа
            x1, y1, x2, y2, conf, cls = box
            # Преобразуем координаты в целые числа, если это еще не сделано
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            # Рисуем прямоугольник на изображении
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # Опционально: добавление текста к bounding box
            cv2.putText(image, f'Conf: {conf:.2f}, Class: {cls}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
    return image

In [34]:
# Рисование bounding boxes на изображении
# test_detections = {'boxes': [(100, 100, 200, 200, 0.9, 1)]}
image_with_boxes = draw_boxes(image, detections)

# Конвертация обратно в формат, пригодный для отправки через HTTP
_, buffer = cv2.imencode('.jpg', image_with_boxes)
img_str = base64.b64encode(buffer).decode('utf-8')

json_res = JSONResponse(content={"image": img_str})

In [35]:
json_res